In [2]:
import yfinance as yf
import math
import pandas as pd
import numpy as np

In [3]:
def BASS(stock_symbol, benchmark_symbol, start, end):

  # prep data with returns
  df = pd.DataFrame()
  df["benchmark"] = yf.Ticker(benchmark_symbol).history(start=start, end=end).Close
  df["stock"] = yf.Ticker(stock_symbol).history(start=start, end=end).Close
  df["benchmark_returns"] = df["benchmark"].pct_change()
  df["stock_returns"] = df["stock"].pct_change()
  df = df.dropna()

  # Calculate Beta
  cov = df['benchmark_returns'].cov(df['stock_returns'])
  var = df['benchmark_returns'].var()
  beta = cov/var
  beta = round(beta, 2)
  # print("The beta is {:.2f}".format(beta))

  # Calculate Alpha
  benchmark_yearly_returns = (df["benchmark_returns"].mean()*252)
  stock_yearly_returns = (df["stock_returns"].mean()*252)
  alpha = (stock_yearly_returns - beta * benchmark_yearly_returns)*100
  alpha = round(alpha, 2)
  # print("The alpha is {:.2f}".format(alpha))

  # Calculate SD
  std_dev = (df["stock_returns"].std())*100
  std_dev = round(std_dev, 2)
  # print("The standard deviation is {:.2f}".format(std_dev))

  # Calculate Sharpe Ratio
  avg_returns = df["stock_returns"].mean()
  std = df["stock_returns"].std()
  daily_SR = avg_returns / std
  annual_SR = daily_SR * (252**0.5)
  annual_SR = round(annual_SR, 2)
  # print("The Sharpe Ratio is {:.2f}".format(annual_SR))

  return beta, alpha, std_dev, annual_SR

In [8]:
all_stocks = ["C52.SI", "S68.SI", "G13.SI", "V03.SI" , "U11.SI",
              "C07.SI" , "D05.SI", "Z74.SI", "D01.SI", "O39.SI", "S63.SI",
              "A17U.SI" , "BN4.SI","BS6.SI", "M44U.SI", "H78.SI", "Y92.SI",
              "C38U.SI", "U14.SI", "N2IU.SI" , "F34.SI" , "C09.SI" , "J36.SI",
              "S58.SI" , "C6L.SI", "U96.SI" , "1810.HK", "9999.HK", "7500.HK",
              "9618.HK", "1024.HK", "3690.HK", "6618.HK"]

stocks, betas, alphas, std_devs, sratios = [], [], [], [], []
benchmark_symbol = "^STI"
start = "2021-01-01"
end = "2021-12-31"

In [9]:
for i in all_stocks:
  try:
    BASS_output = BASS(i, benchmark_symbol, start, end)
    betas.append(BASS_output[0])
    alphas.append(BASS_output[1])
    std_devs.append(BASS_output[2])
    sratios.append(BASS_output[3])
    stocks.append(i)
  except:
    print('The symbol {} not found'.format(i))

# prep output
output_df = pd.DataFrame()
output_df['stock'] = stocks
output_df['beta'] = betas
output_df['alpha(%)'] = alphas
output_df['std_devs(%)'] = std_devs
output_df['sharpe'] = sratios

print(output_df.sort_values(by='sharpe', ascending=False))

      stock  beta  alpha(%)  std_devs(%)  sharpe
6    D05.SI  1.28     18.03         1.12    1.71
4    U11.SI  1.08     11.93         0.93    1.52
13   BS6.SI  0.57     35.27         1.80    1.43
15   H78.SI  0.59     26.67         1.43    1.42
9    O39.SI  1.23      4.69         0.98    1.06
25   U96.SI  1.16     10.33         1.68    0.81
24   C6L.SI  1.36      5.99         1.71    0.70
28  7500.HK -1.30     37.93         2.52    0.60
5    C07.SI  0.81      2.18         1.38    0.46
27  9999.HK  1.40      0.49         3.26    0.30
7    Z74.SI  0.95     -4.35         1.07    0.28
11  A17U.SI  0.66     -2.89         0.92    0.24
22   J36.SI  1.11     -4.87         1.78    0.21
10   S63.SI  0.76     -4.90         0.90    0.17
14  M44U.SI  0.75     -6.37         1.02    0.05
1    S68.SI  0.75     -6.53         1.04    0.04
23   S58.SI  0.91     -8.37         1.25    0.02
12   BN4.SI  1.10    -11.01         1.27   -0.02
17  C38U.SI  0.88     -8.85         1.12   -0.02
19  N2IU.SI  0.94   